In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

TensorFlow version: 2.9.2


In [5]:
train_dir="/content/drive/MyDrive/cbis_ddsm/mass/train"
test_dir="/content/drive/MyDrive/cbis_ddsm/mass/test"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir, color_mode="grayscale",image_size=(227,227), batch_size=10)
# test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir, color_mode="grayscale",image_size=(227,227), batch_size=10)

Found 552 files belonging to 3 classes.


In [24]:
def FCN_model(len_classes=3,shape=(227, 227, 1), dropout_rate=0.2):
    
    input = tf.keras.layers.Input(shape=shape)

    x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1,padding="valid")(input)
    x= tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    x = tf.keras.layers.MaxPooling2D(2)(x)

    x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    x = tf.keras.layers.MaxPooling2D(2)(x)

    x = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=2)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    x = tf.keras.layers.MaxPooling2D(2)(x)

    x = tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=2)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    x = tf.keras.layers.MaxPooling2D(2)(x)

    x = tf.keras.layers.Conv2D(filters=512, kernel_size=3, strides=2)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    
    # Fully connected layer 1
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=1, strides=1)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # Fully connected layer 2
    x = tf.keras.layers.Conv2D(filters=len_classes, kernel_size=1, strides=1)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.GlobalMaxPooling2D()(x)
    predictions = tf.keras.layers.Activation('softmax')(x)

    model = tf.keras.Model(inputs=input, outputs=predictions)
    
    print(model.summary())
    print(f'Total number of layers: {len(model.layers)}')

    return model

In [25]:
fcn0=FCN_model()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 227, 227, 1)]     0         
                                                                 
 conv2d_28 (Conv2D)          (None, 225, 225, 32)      320       
                                                                 
 dropout_28 (Dropout)        (None, 225, 225, 32)      0         
                                                                 
 batch_normalization_28 (Bat  (None, 225, 225, 32)     128       
 chNormalization)                                                
                                                                 
 activation_28 (Activation)  (None, 225, 225, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                         

In [26]:
fcn0.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),#RMSprop(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [27]:
# fcn0.fit(x_train, y_train,epochs=5)
history=fcn0.fit(train_data,epochs=20)

Epoch 1/20
56/56 [==============================] - 113s 2s/step - loss: 1.3910 - accuracy: 0.2989
Epoch 2/20
56/56 [==============================] - 118s 2s/step - loss: 1.1595 - accuracy: 0.4022
Epoch 3/20
56/56 [==============================] - 112s 2s/step - loss: 1.1670 - accuracy: 0.4094
Epoch 4/20
56/56 [==============================] - 110s 2s/step - loss: 1.0938 - accuracy: 0.4203
Epoch 5/20
56/56 [==============================] - 111s 2s/step - loss: 1.1740 - accuracy: 0.4004
Epoch 6/20
56/56 [==============================] - 110s 2s/step - loss: 1.1067 - accuracy: 0.4402
Epoch 7/20
15/56 [=======>......................] - ETA: 1:21 - loss: 1.1110 - accuracy: 0.4467

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
acc = history.history['accuracy']
# val_acc = history.history['val_accuracy']

loss = history.history['loss']
# val_loss = history.history['val_loss']

epochs_range = range(5)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
# plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
# plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


In [ ]:
keras.utils.plot_model(fcn0, to_file='model.png', show_shapes=True, show_dtype=True, show_layer_names=True, rankdir='TB', expand_nested=True, dpi=96, layer_range=None, show_layer_activations=True)

Test the model: